In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PyQt6 import QtWidgets

plt.rcParams["backend"] = 'QtAgg'
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["figure.dpi"] = 250
plt.rcParams["savefig.format"] = "svg"
plt.rcParams['animation.frame_format'] = "svg"
plt.rcParams['font.family'] = 'serif'

import sys
import os
wd = %pwd
parent = os.path.dirname(wd)
fdel = os.path.sep

print(parent)
sys.path.append(parent)

import nanoscale_measurement as nano

/home/purpy-furcat/Documents/Code/Lab_Related


In [7]:
thickness = nano.profilometry.main.measure_thickness()



 proflometry module by James Bishop, use for analysis of profilometry trace data from DekTak profilomoter. Email: james.bishop@uts.edu.au if you have further questions. 


Remove outliers method,  Delta %:  10
Adding vertical line for measurement at x =  0.592132349587921
Adding vertical line for measurement at x =  1.9815855147349697
Will fit substratself.all_datae baseline for leveling between, x1: 0.592132349587921, x2: 1.9815855147349697
X1 and X2 set:  0.592132349587921 1.9815855147349697
Fitting line to substrate between x1 and x2 using linear regression (np.polyfit(deg = 1)).
          Is a LBF with equal weighting to all points between x1 & x2.
Adding vertical line for measurement at x =  0.007763242309737128
Will fit substratself.all_datae baseline for leveling between, x1: 0.592132349587921, x2: 1.9815855147349697


In [14]:
thickness_measure = thickness[0]
pro_trace = thickness[1]
pro_trace.data
pro_trace.leveled_data.plot()
plt.show()

<img src="/home/purpy-furcat/Documents/Code/Lab_Related/Traces/salma_plot.svg" alt="Despiked, leveled, interpolated profilometry data." />

In [15]:
def qt_load_file_dialog(dialog_title: str = "Choose a file", initial_dir: str = wd,
                        file_types: str = "All Files (*);;Text Files (*.txt);;Excel Files (*.xlsx);;csv Files (*.csv)",):

    app = QtWidgets.QApplication.instance()  # Check if an instance already exists

    if not app:  # If not, create a new instance
        app = QtWidgets.QApplication(sys.argv)

    file_path, _ = QtWidgets.QFileDialog.getOpenFileName(None, dialog_title, initial_dir, file_types, options=QtWidgets.QFileDialog.Option.DontUseNativeDialog)

    return file_path

def profilometry_plot(data: pd.Series, **kwargs):
    fig, ax = plt.subplots(1, 1)
    ax.plot(data)
    ax.axhline(y=0, color='r', linestyle='--')
    ax.grid(visible=True, which='both', axis='both', ls = ":", lw = 0.75)
    # Increase the number of ticks on x and y axes
    ax.minorticks_on()
    ax.set_xlabel("x (mm)")
    ax.set_ylabel("Height (nm)")
    ax.set_title("Profilometry data")
    # ax.xaxis.set_major_locator(MaxNLocator(nbins=19))
    # ax.yaxis.set_major_locator(MaxNLocator(nbins=19))
    return fig, ax   


def find_closest_index(series: pd.Series, target_index_value: float) -> tuple:
    # Subtract the target value from the series and take the absolute value
    diff = abs((series.index - target_index_value))
    # Find the index of the minimum value in the resulting series
    closest_iloc = diff.argmin()
    # Find the iloc position of the closest index
    closest_index_value = series.index[closest_iloc]
    return closest_index_value, closest_iloc
